In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import adjusted_rand_score
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor, XGBRFRegressor
from catboost import CatBoostRegressor

In [2]:
X_train = pd.read_csv("./data/Train.csv")
X_test = pd.read_csv("./data/Test.csv")

In [3]:
X_train

,id,date,street_id,build_tech,floor,area,rooms,balcon,metro_dist,g_lift,...,kw5,kw6,kw7,kw8,kw9,kw10,kw11,kw12,kw13,price
0,0,2011-1,560,NaN,2,59,3,0,30.0,1.0,...,0,0,0,0,0,0,0,0,0,4510000
1,1,2011-1,667,NaN,10,50,2,1,25.0,NaN,...,0,0,0,0,0,0,0,0,0,13231000
2,2,2011-1,90,0.0,1,48,2,0,25.0,0.0,...,0,0,0,0,0,0,0,0,0,2008000
3,3,2011-1,94,1.0,3,62,3,1,30.0,NaN,...,0,0,0,0,0,0,0,0,0,12680000
4,4,2011-1,232,0.0,3,60,3,0,25.0,NaN,...,0,0,0,0,0,0,0,0,0,3335000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99995,2012-3,401,0.0,5,30,1,0,30.0,NaN,...,0,0,0,0,0,0,0,0,0,1697000
99996,99996,2012-3,59,0.0,4,34,1,0,30.0,NaN,...,0,0,0,0,0,0,0,0,0,3620000
99997,99997,2012-3,253,NaN,3,52,2,0,5.0,NaN,...,0,0,0,0,0,0,0,0,0,6712000
99998,99998,2012-3,305,1.0,3,181,5,1,5.0,NaN,...,0,0,0,0,0,0,0,0,0,20835000


In [4]:
X_test

,id,date,street_id,build_tech,floor,area,rooms,balcon,metro_dist,g_lift,...,kw4,kw5,kw6,kw7,kw8,kw9,kw10,kw11,kw12,kw13
0,100000,2012-3,459,NaN,1,60,3,1,30.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,100001,2012-3,344,1.0,10,52,2,1,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2,100002,2012-3,585,0.0,4,54,3,0,30.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,100003,2012-3,494,NaN,2,52,2,1,25.0,1.0,...,0,0,0,0,0,0,0,0,0,0
4,100004,2012-3,622,1.0,9,60,3,1,15.0,NaN,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,199995,2013-9,613,1.0,2,34,1,1,25.0,NaN,...,0,0,0,0,0,0,0,0,0,0
99996,199996,2013-9,167,1.0,10,61,3,1,30.0,NaN,...,0,0,0,0,0,0,0,0,0,0
99997,199997,2013-9,391,1.0,9,34,1,1,30.0,NaN,...,0,0,0,0,0,0,0,0,0,0
99998,199998,2013-9,21,0.0,7,58,3,0,30.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
y_train = X_train["price"]

# Предобработка признаков 

In [6]:
corr = X_train.corr()
corr.style.background_gradient()

,id,street_id,build_tech,floor,area,rooms,balcon,metro_dist,g_lift,n_photos,kw1,kw2,kw3,kw4,kw5,kw6,kw7,kw8,kw9,kw10,kw11,kw12,kw13,price
id,1.000000,0.000245,-0.002691,0.000173,-0.000767,-0.002868,-0.002670,0.004035,-0.006078,-0.000456,0.000561,0.006195,-0.005825,0.004147,0.002311,-0.000646,-0.000417,-0.005935,0.000547,0.003901,0.000012,0.002174,0.004167,0.057665
street_id,0.000245,1.000000,-0.013550,-0.009430,-0.011883,0.002162,-0.009412,0.009849,-0.005105,0.007101,-0.002139,0.001743,0.000157,0.003503,0.005246,-0.006397,-0.000589,-0.000272,0.007218,-0.003298,0.004564,-0.002310,0.004244,-0.023578
build_tech,-0.002691,-0.013550,1.000000,0.180328,0.322652,0.126698,0.787092,0.016827,0.005915,0.026605,-0.000295,-0.026422,-0.080369,-0.073738,-0.058002,-0.036239,-0.027226,0.073463,0.072732,0.039033,0.014171,0.091782,0.030576,0.369538
floor,0.000173,-0.009430,0.180328,1.000000,0.150671,-0.001009,0.143858,0.013032,0.003609,0.003441,-0.001135,-0.038333,-0.049537,-0.004778,0.025233,0.007706,0.070087,-0.053204,0.026793,0.012333,0.004214,0.064565,0.002364,0.201177
area,-0.000767,-0.011883,0.322652,0.150671,1.000000,0.791829,0.256769,0.005431,0.001387,0.170953,-0.000854,-0.062135,-0.041959,-0.019717,-0.014830,0.011360,0.005579,0.001342,-0.003729,0.096234,0.052317,0.022090,0.039629,0.662661
rooms,-0.002868,0.002162,0.126698,-0.001009,0.791829,1.000000,0.100746,0.003746,0.000867,0.213999,0.000186,-0.008418,-0.012525,-0.010019,-0.005441,-0.003180,-0.007141,0.014303,0.006452,0.026909,0.017657,-0.006712,0.009572,0.377233
balcon,-0.002670,-0.009412,0.787092,0.143858,0.256769,0.100746,1.000000,0.014015,0.008503,0.020215,0.002481,-0.023451,-0.061997,-0.057685,-0.047942,-0.027416,-0.022222,0.054662,0.058816,0.033088,0.013583,0.065131,0.034869,0.292242
metro_dist,0.004035,0.009849,0.016827,0.013032,0.005431,0.003746,0.014015,1.000000,-0.000164,0.008822,0.003819,0.000972,-0.003879,0.002529,0.000742,0.004757,-0.002095,0.002466,0.001748,-0.002424,-0.000829,0.002011,-0.000875,-0.092270
g_lift,-0.006078,-0.005105,0.005915,0.003609,0.001387,0.000867,0.008503,-0.000164,1.000000,0.002694,0.001952,-0.009085,-0.006320,-0.005286,-0.001012,-0.006249,-0.000542,-0.001146,-0.002386,0.000517,-0.005105,-0.004854,-0.001075,0.002752
n_photos,-0.000456,0.007101,0.026605,0.003441,0.170953,0.213999,0.020215,0.008822,0.002694,1.000000,0.037015,-0.014125,0.000463,0.005935,0.007233,0.002267,0.004989,0.009084,0.009499,0.000479,0.004005,0.004440,0.002878,0.189277


Для начала удалим колонки "id" и "date". Выборка обЪектов лежит в узком промежутке времени, поэтому можно предположить, что этот признак не влияет на цену. Также удалим признаки "kw8" и "kw9", так как у них очень низкая корреляция с ценой.

In [7]:
X_train.drop(columns=["id", "date", "price"], inplace=True)

In [8]:
X_test.drop(columns=["id", "date"], inplace=True)

In [9]:
X_train.drop(columns=["kw8", "kw9"], inplace=True)
X_test.drop(columns=["kw8", "kw9"], inplace=True)

В признаке "g_lift" есть пропуски данных, и он имеет низкую корреляцию с ценой, поэтому его можно удалить.

In [10]:
del X_train["g_lift"]
del X_test["g_lift"]

### Заполнение пропущенных данных

In [11]:
X_train["metro_dist"] = X_train["metro_dist"].fillna(X_train["metro_dist"].mean())
X_test["metro_dist"] = X_test["metro_dist"].fillna(X_test["metro_dist"].mean())

Для признака "build_tech" нельзя однозначно определить значение, которым можно заполнить пропуски. Поэтому найдем эти значения обучив RandomForestClassifier на данных где нет пропусков.

In [12]:
X_bt_train = X_train[X_train["build_tech"].notnull()].drop(columns="build_tech")

y_bt_train = X_train[X_train["build_tech"].notnull()]["build_tech"].astype(int).values

In [13]:
RFC = RandomForestClassifier(n_estimators=100)

In [14]:
RFC.fit(X_bt_train, y_bt_train)

RandomForestClassifier()

In [15]:
adjusted_rand_score(y_bt_train, RFC.predict(X_bt_train))

0.9990114616887225

In [16]:
bt_pred = pd.Series(RFC.predict(X_train.drop(columns="build_tech")))

In [17]:
X_train.build_tech.fillna(bt_pred, inplace=True)

In [18]:
bt_test_pred = RFC.predict(X_test.drop(columns="build_tech"))

In [19]:
X_test.build_tech.fillna(pd.Series(bt_test_pred), inplace=True)

# RandomForest

In [20]:
RFR = RandomForestRegressor(n_estimators=500, max_depth=None)

In [21]:
RFR.fit(X_train, y_train)

RandomForestRegressor(n_estimators=500)

In [22]:
pred = RFR.predict(X_train)

In [23]:
mse(y_train, pred)

775523600712.0187

In [24]:
pred

array([ 4264338., 12581788.,  2454520., ...,  6569832., 24576124.,
        3061272.])

1. 807 400 846 719.8564: с streat_id
2. 2 902 489 028 991.333: без streat_id
3. 2 644 326 445 087.0024: build_tech fill with 0
4. 2 764 680 523 813.0986: build_tech fill with 1
5. 833 302 418 286.274: with streat_id, build_tech fill with 1
6. 849 381 531 680.5408: with streat_id, build_tech fill with 0
7. 804 654 005 031.6157: with streat_id, build_tech predicted
8. 774 911 468 522.5715: 7 + max_depth=None

# GradientBoosting

In [25]:
GBC = GradientBoostingRegressor(n_estimators=500, learning_rate=1., max_depth=None)

In [ ]:
GBC.fit(X_train, y_train)

In [ ]:
pred = GBC.predict(X_train)

In [ ]:
mse(y_train, pred)

1. 7 265 309 555.000001: with streat_id, build_tech predicted

# LGBMRegressor

In [ ]:
LGBM = LGBMRegressor(n_estimators=500, learning_rate=1., max_depth=None)

In [ ]:
LGBM.fit(X_train, y_train)

In [ ]:
pred = LGBM.predict(X_train)

In [ ]:
mse(y_train, pred)

1. 3 636 727 300 694.436
2. 1 989 393 629 633.3767


# XGBRegressor

In [ ]:
XG = XGBRegressor(n_estimators=500,  learning_rate=1., max_depth=None)

In [ ]:
XG.fit(X_train, y_train)

In [ ]:
pred = XG.predict(X_train)

In [ ]:
mse(y_train, pred)

1. 696 929 541 556.401

# XGBRFRegressor

In [ ]:
XGFR = XGBRFRegressor(n_estimators=500,  learning_rate=1., max_depth=None)

In [ ]:
XGFR.fit(X_train, y_train)

In [ ]:
pred = XGFR.predict(X_train)

In [ ]:
mse(y_train, pred)

1. 7 843 171 087 396.351

# CatBoostRegressor

In [ ]:
CB = CatBoostRegressor(n_estimators=500,  learning_rate=1., max_depth=None)

In [ ]:
CB.fit(X_train, y_train)

In [ ]:
pred = CB.predict(X_train)

In [ ]:
mse(y_train, pred)

1. 2 979 644 213 646.3584